<a href="https://colab.research.google.com/github/jumbokh/nknu-class/blob/main/CNN/notebooks/07_02_%E5%9C%96%E5%83%8F%E7%9B%B8%E4%BC%BC%E5%BA%A6%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 圖像相似度比較

## 載入套件

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -fs /content/gdrive/My\ Drive /app
%cd /app/DL_Book/src

Mounted at /content/gdrive
/content/gdrive/My Drive/DL_Book/src


In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

## 載入VGG 16 模型

In [3]:
# 載入VGG 16 模型, 不含最上面的三層(辨識層)
model = VGG16(weights='imagenet', include_top=False)
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [4]:
# 任選一張圖片，例如大象側面照，取得圖檔的特徵向量
img_path = '../images_test/elephant.jpg'

# 載入圖檔，並縮放寬高為 (224, 224) 
img = image.load_img(img_path, target_size=(224, 224))

# 加一維，變成 (1, 224, 224)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# 取得圖檔的特徵向量
features = model.predict(x)
print(features[0])

[[[ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.        42.547764  ...  0.         0.         0.       ]
  [ 1.0753386  0.        23.49565   ...  0.         0.         0.       ]
  ...
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]]

 [[ 0.         0.        36.338863  ...  0.         0.         3.40288  ]
  [ 0.         0.        80.23628   ...  7.8718963  0.         0.       ]
  [ 0.         0.        48.75137   ...  0.         0.         0.       ]
  ...
  [ 0.         0.         0.        ...  4.581369   0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]
  [ 0.         0.         0.        ...  0.         0.         0.       ]]

 [[ 0.         0.         9.85349   ...  0.         0.         2.491967 ]
  [ 0.         0.     

In [5]:
print(features.shape)

(1, 7, 7, 512)


# 使用 cosine_similarity 比較特徵向量

### 步驟 1. 取得 images_test 目錄下所有 .jpg 檔案名稱

In [6]:
from os import listdir
from os.path import isfile, join

# 取得 images_test 目錄下所有 .jpg 檔案名稱
img_path = '../images_test/'
image_files = np.array([f for f in listdir(img_path) 
        if isfile(join(img_path, f)) and f[-3:] == 'jpg'])
image_files

array(['elephant2.jpg', 'elephant.jpg', 'tiger2.jpg', 'tiger1.jpg',
       'tiger3.jpg', 'bird.jpg', 'lion1.jpg', 'deer.jpg', 'bird2.jpg',
       'lion2.jpg', 'panda1.jpg', 'panda2.jpg', 'panda3.jpg', 'rose2.jpg',
       'daisy1.jpg', 'daisy2.jpg'], dtype='<U13')

### 步驟 2. 取得 images_test 目錄下所有 .jpg 檔案的像素

In [7]:
import numpy as np

# 合併所有圖檔的像素
X = np.array([])
for f in image_files:
    image_file = join(img_path, f)
    # 載入圖檔，並縮放寬高為 (224, 224) 
    img = image.load_img(image_file, target_size=(224, 224))
    img2 = image.img_to_array(img)
    img2 = np.expand_dims(img2, axis=0)
    if len(X.shape) == 1:
        X = img2
    else:
        X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)

### 步驟 3. 取得所有圖檔的特徵向量

In [8]:
# 取得所有圖檔的特徵向量
features = model.predict(X)

features.shape, X.shape

((16, 7, 7, 512), (16, 224, 224, 3))

### 步驟 4. 使用 cosine_similarity 函數比較特徵向量

In [9]:
# 使用 cosine_similarity 比較特徵向量
from sklearn.metrics.pairwise import cosine_similarity


# 比較 Tiger2.jpg 與其他圖檔特徵向量
no=-2
print(image_files[no])

# 轉為二維向量，類似扁平層(Flatten)
features2 = features.reshape((features.shape[0], -1))

# 排除 Tiger2.jpg 的其他圖檔特徵向量
other_features = np.concatenate((features2[:no], features2[no+1:]))

# 使用 cosine_similarity 計算 Cosine 函數
similar_list = cosine_similarity(features2[no:no+1], other_features, 
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

daisy1.jpg
[0.19570272 0.05166131 0.03469756 0.03114462 0.03097638 0.0268551
 0.02586457 0.02431086 0.02324399 0.02200264 0.02107495 0.02007598
 0.0199604  0.01920781 0.0162836 ]


array(['daisy2.jpg', 'rose2.jpg', 'panda1.jpg', 'tiger2.jpg',
       'elephant2.jpg', 'panda3.jpg', 'bird2.jpg', 'lion1.jpg',
       'tiger1.jpg', 'elephant.jpg', 'tiger3.jpg', 'lion2.jpg',
       'deer.jpg', 'panda2.jpg', 'bird.jpg'], dtype='<U13')

### 其他圖檔比較

In [10]:
# 比較對象：bird.jpg
no=0
print(image_files[no])


# 使用 cosine_similarity 計算 Cosine 函數
other_features = np.concatenate((features2[:no], features2[no+1:]))
similar_list = cosine_similarity(features2[no:no+1], other_features, 
                                 dense_output=False)

# 顯示相似度，由大排到小
print(np.sort(similar_list[0])[::-1])

# 依相似度，由大排到小，顯示檔名
image_files2 = np.delete(image_files, no)
image_files2[np.argsort(similar_list[0])[::-1]]

elephant2.jpg
[0.29506835 0.18226714 0.16894677 0.10598635 0.09735928 0.07993932
 0.07924502 0.0768037  0.07652343 0.07266308 0.07242106 0.06952199
 0.06780844 0.0421958  0.03097638]


array(['elephant.jpg', 'lion1.jpg', 'tiger2.jpg', 'rose2.jpg',
       'tiger1.jpg', 'bird.jpg', 'bird2.jpg', 'panda2.jpg', 'panda3.jpg',
       'panda1.jpg', 'tiger3.jpg', 'deer.jpg', 'lion2.jpg', 'daisy2.jpg',
       'daisy1.jpg'], dtype='<U13')